In [1]:
import tensorflow as tf
import os

In [2]:
FLAGS=tf.app.flags.FLAGS

In [3]:
tf.app.flags.DEFINE_string("captcha_dir", "./data/captcha/captcha.tfrecords", "验证码数据路径")

In [4]:
tf.app.flags.DEFINE_integer("batch_size",100,"每批次训练的样本数")
tf.app.flags.DEFINE_integer("label_num", 4,"每个样本的目标值数量")
tf.app.flags.DEFINE_integer("letter_num", 26,"每个目标值的字母的可能性个数")
tf.app.flags.DEFINE_string("f", "","fix bug")


In [5]:
def read_and_decode():
    """
    读取验证码数据API
    return: image_batch=[100,20,80,3], label_batch=[100, 4]
    """
    # 1. 构建文件队列
    file_queue = tf.train.string_input_producer([FLAGS.captcha_dir])
    
    # 2. 构建文件阅读器、读取文件内容，默认读取一个样本
    reader = tf.TFRecordReader()
    
    # 3. 读取内容,key: filename,
    key, value = reader.read(file_queue)
    
    # 4. parse tfrecords example
    features = tf.parse_single_example(value, features={
        "image":tf.FixedLenFeature([], tf.string),
        "label":tf.FixedLenFeature([], tf.string)
    })
    
    # 解码内容，字符串内容
    # 解析图片的特征值
    image = tf.decode_raw(features["image"], tf.uint8)

    # 解析图片的目标值
    label = tf.decode_raw(features["label"], tf.uint8)
    
    print(image, label)
    # 改变形状
    image_reshape = tf.reshape(image, [20, 80, 3])
    label_reshape = tf.reshape(label, [4])
    
    print(image_reshape, label_reshape)
    
    # 批处理，每次读取的样本数 100， 即每次训练的时候的样本
    image_batch, label_batch = tf.train.batch([image_reshape, label_reshape],
                                             batch_size=FLAGS.batch_size,
                                             num_threads=1,
                                             capacity=FLAGS.batch_size) 
    
    print(image_batch,label_batch)
    return image_batch, label_batch

In [6]:
def fc_model(image):
    """
    image: 100 images, features [100, 20, 80, 3]
    return: y_predict [100, 4*26]
    """
    with tf.variable_scope("model"):
        # 1. input reshape
        image_reshape = tf.reshape(image, [-1, 20*80*3])
        
        # 2. init weights [20*80*3, 4*26] bias=[4*26]
        # matrix[100, 20 * 80 * 3] * [20 * 80 * 3, 4 * 26] + [104] = [100, 4 * 26]
        weights = tf.Variable(tf.random_normal([20*80*3, 4*26], mean=0.0, stddev=1.0))
        bias = tf.Variable(tf.constant(0.0,shape=[4*26]))
        
        # 3. full connect layer calcuate 
        y_predict = tf.matmul(tf.cast(image_reshape, tf.float32), weights) + bias
    return y_predict 

In [7]:
# def predict_to_onehot(label):
    """
    将读取文件当中的目标值转换成one_hot编码
    label:[100, 4] [[13,25,15,15],[19,23,20,16]......]
    return: [100, 4, 26]
    """
    # 1. 

'\n            将读取文件当中的目标值转换成one_hot编码\n            label:[100, 4] [[13,25,15,15],[19,23,20,16]......]\n            return: [100, 4, 26]\n            '

In [8]:
# def captcha_rec():

# 1. 读取验证码的数据文件  image_batch=[100,20,80,3], label_batch=[100, 4] 
image_batch, label_batch = read_and_decode()
# print(image_batch, label_batch)

# 2. 通过输入图片特征, 建立模型，得出预测结果
# 一层 全连接神经网络进行预测
# matric [100, 20*80*3] * [20*80*3, 4*26] + [4*26] = [100, 4*26]
y_predict = fc_model(image_batch)
print("y_predict:",y_predict)

# 3. 把真实的目标值转换成one-hot编码 
# label_batch:[100, 4] [[13,25,15,15],[19,23,20,16]......] 
# [100, 4] -> [100, 4, 26]
y_true = tf.one_hot(label_batch, depth=26, on_value=1.0, axis=2)
print("y_true:", y_true)

# 4. softmax计算，交叉熵损失计算 二维计算损失
with tf.variable_scope("soft_cross"):
    # 求平均交叉熵损失
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.reshape(y_true,[100,4*26]),
        logits=y_predict))
    print(loss)

# 5. 梯度下降优化损失
with tf.variable_scope("optimizer"):
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    
# 6. 求出样本每批次的准确率是多少， 三维比较 
with tf.variable_scope("optimizer"):
    # 比较每个预测值和目标值是否位置（4个位置）一样
    # y_true [100, 4, 26]  y_predict [100, 4*26] -> [100, 4, 26]
    # equal_list_2d [100, 4] 
    equal_list_2d = tf.equal(
        tf.argmax(y_true,axis=2),
        tf.argmax(tf.reshape(y_predict,[100,4,26]),axis=2)
    )
    print(equal_list_2d)
    
    # equal_list:[100,4] -> reduce_mean(axis=1) -> [100] -> reduce_mean() -> scalar 
    result_list_1d = tf.reduce_mean(
        tf.cast(equal_list_2d, tf.int8),
        axis=1
    )
    print(result_list_1d)
    
    accuracy = tf.reduce_mean(tf.cast(result_list_1d, tf.float32))
    print(accuracy)
    


W0213 04:19:03.191406 140087226820416 deprecation.py:323] From <ipython-input-5-739332c3d1a7>:7: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0213 04:19:03.297010 140087226820416 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/input.py:278: input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0213 

Tensor("DecodeRaw:0", shape=(?,), dtype=uint8) Tensor("DecodeRaw_1:0", shape=(?,), dtype=uint8)
Tensor("Reshape:0", shape=(20, 80, 3), dtype=uint8) Tensor("Reshape_1:0", shape=(4,), dtype=uint8)
Tensor("batch:0", shape=(100, 20, 80, 3), dtype=uint8) Tensor("batch:1", shape=(100, 4), dtype=uint8)
y_predict: Tensor("model/add:0", shape=(100, 104), dtype=float32)
y_true: Tensor("one_hot:0", shape=(100, 4, 26), dtype=float32)
Tensor("soft_cross/Mean:0", shape=(), dtype=float32)
Tensor("optimizer_1/Equal:0", shape=(100, 4), dtype=bool)
Tensor("optimizer_1/Mean:0", shape=(100,), dtype=int8)
Tensor("optimizer_1/Mean_1:0", shape=(), dtype=float32)


In [9]:
# 定义一个初始化变量的op
init_op = tf.global_variables_initializer()

# 定义一个保存模型的实例
saver = tf.train.Saver(max_to_keep=5)

# 开启会话训练
with tf.Session() as sess:
    # 初始化变量
    sess.run(init_op)
    
    # 加载模型
    if os.path.exists("./ckpt/captcha/captcha_model.data-00000-of-00001"):
        saver.restore(sess, "./ckpt/captcha/captcha_model")
        print("restore!!!")
    
    # 定义线程协调器和开启线程
    coord = tf.train.Coordinator()
    
    # 开启线程去运行读取文件操作
    threads = tf.train.start_queue_runners(sess, coord=coord)
    
#     print(sess.run(y_true))
#     print(sess.run(equal_list_2d))
#     print(sess.run(result_list_1d))
    # 循环训练
    for i in range(5000):
        sess.run(train_op)
#         print("%d batchs accuracy=%f" % (i, sess.run(accuracy)))
        
        if i%100 == 0:
            saver.save(sess, "./ckpt/captcha/captcha_model")
            print("%d batchs accuracy=%f" % (i, sess.run(accuracy)))
    
    # 回收线程
    coord.request_stop()
    coord.join()

W0213 04:19:03.769759 140087226820416 deprecation.py:323] From <ipython-input-9-e0c006ab9ddc>:21: start_queue_runners (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


0 batchs accuracy=0.000000
100 batchs accuracy=0.000000
200 batchs accuracy=0.040000
300 batchs accuracy=0.200000
400 batchs accuracy=0.480000
500 batchs accuracy=0.560000
600 batchs accuracy=0.720000
700 batchs accuracy=0.830000
800 batchs accuracy=0.880000
900 batchs accuracy=0.850000
1000 batchs accuracy=0.820000
1100 batchs accuracy=0.920000
1200 batchs accuracy=0.900000
1300 batchs accuracy=0.900000
1400 batchs accuracy=0.890000
1500 batchs accuracy=0.800000
1600 batchs accuracy=0.900000
1700 batchs accuracy=0.960000
1800 batchs accuracy=0.950000
1900 batchs accuracy=0.900000
2000 batchs accuracy=0.900000
2100 batchs accuracy=0.940000
2200 batchs accuracy=0.950000
2300 batchs accuracy=0.920000
2400 batchs accuracy=0.950000
2500 batchs accuracy=0.970000
2600 batchs accuracy=0.850000
2700 batchs accuracy=0.970000
2800 batchs accuracy=0.870000
2900 batchs accuracy=0.890000
3000 batchs accuracy=0.950000
3100 batchs accuracy=0.900000
3200 batchs accuracy=0.960000
3300 batchs accuracy=0

In [10]:
import os

In [11]:
os.path.exists("./ckpt/captcha/captcha_model.data-00000-of-00001")

True

Help on method save in module tensorflow.python.training.saver:

save(sess, save_path, global_step=None, latest_filename=None, meta_graph_suffix='meta', write_meta_graph=True, write_state=True, strip_default_attrs=False, save_debug_info=False) method of tensorflow.python.training.saver.Saver instance
    Saves variables.
    
    This method runs the ops added by the constructor for saving variables.
    It requires a session in which the graph was launched.  The variables to
    save must also have been initialized.
    
    The method returns the path prefix of the newly created checkpoint files.
    This string can be passed directly to a call to `restore()`.
    
    Args:
      sess: A Session to use to save the variables.
      save_path: String.  Prefix of filenames created for the checkpoint.
      global_step: If provided the global step number is appended to `save_path`
        to create the checkpoint filenames. The optional argument can be a
        `Tensor`, a `Tensor` nam

In [13]:

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess, coord=coord)
    
    print(sess.run(features["label"]))
    print(sess.run(label))
    
    coord.request_stop()
    coord.join(threads)

NameError: name 'features' is not defined

In [ ]:
a = b'\r\x19\x0f\x0f'
b = b'\r\n'
for i in b:
    
    print(i)